In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
import spacv
from spacv.grid_builder import *
from spacv.utils import *
import unittest

from sklearn.cluster import KMeans, MiniBatchKMeans

C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\geopandas\_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(
Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_KEA.dll
127: The specified procedure could not be found.



# Sklearn integration

In [2]:
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.svm import SVC

In [3]:
df = gpd.read_file('data/baltim.geojson')

XYs = df['geometry']
X = df[['NROOM', 'BMENT', 'NBATH', 'PRICE', 'LOTSZ', 'SQFT']]
y = df['PATIO']

skcv = spacv.SKCV(n_splits=4, buffer_radius=10).split(XYs)

svc = SVC()

cross_val_score(svc, 
                X, 
                y, 
                cv = skcv)

C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\sklearn\cluster\_kmeans.py:1045: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\spacv\base_classes.py:69: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  deadzone_points = gpd.sjoin(candidate_deadzone, geometry_buffer)
C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\spacv\base_classes.py:69: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  dea

array([0.88372093, 0.85714286, 0.93181818, 0.72222222])

In [4]:
skcv = spacv.SKCV(buffer_radius=5).split(XYs)

parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

clf = GridSearchCV(estimator=svc, 
                   param_grid=parameters, 
                   cv = skcv # An iterable yielding (train, test) splits as arrays of indices
                  )

clf.fit(X, y)

C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\sklearn\cluster\_kmeans.py:1045: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 1024 or by setting the environment variable OMP_NUM_THREADS=1
  warnings.warn(
C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\spacv\base_classes.py:69: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  deadzone_points = gpd.sjoin(candidate_deadzone, geometry_buffer)
C:\Users\lida\Anaconda3\envs\ch\lib\site-packages\spacv\base_classes.py:69: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: None

  dea

GridSearchCV(cv=<generator object BaseSpatialCV.split at 0x00000200EB028580>,
             estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')})

In [5]:
clf.best_params_

{'C': 10, 'kernel': 'rbf'}

### Check data 

In [12]:
XYs

0         POINT (1940.000 2728.000)
1         POINT (2042.000 2462.000)
2          POINT (795.000 2703.000)
3         POINT (1444.000 1507.000)
4         POINT (2791.000 2080.000)
                    ...            
999995    POINT (1791.000 2200.000)
999996      POINT (318.000 518.000)
999997      POINT (312.000 491.000)
999998     POINT (2656.000 704.000)
999999    POINT (2257.000 2873.000)
Name: geometry, Length: 1000000, dtype: geometry

# Performance

In [6]:
n = 1000000

x = np.random.randint(0, 3000, n)
y = np.random.randint(0, 3000, n)

gdf = gpd.GeoDataFrame({'geometry': gpd.points_from_xy(x,y)})
data = np.random.random((n,5))

y, gdf[['V2','V3','V4','V5']] = data[:,0], data[:,1:5]
X = gdf[['V2', 'V3', 'V4', 'V5']].values

XYs = gdf['geometry']

### H-blocking

In [10]:
cv = spacv.HBLOCK(10,5, method='unique', buffer_radius=500)

%timeit
for tr,te in cv.split(XYs):
    pass

In [11]:
cv = spacv.SKCV(folds=10, buffer_radius=500)

%timeit
for tr,te in cv.split(XYs):
    pass

TypeError: __init__() got an unexpected keyword argument 'folds'

### Repeat Spatial K-fold

In [ ]:
cv = spacv.RepeatSKCV(folds=10, n_repeats=2)

%timeit
for tr,te in cv.split(XYs):
    pass

In [ ]:
cv = spacv.RepeatSKCV(folds=10, n_repeats=2, buffer_radius=500)

%timeit
for tr,te in cv.split(XYs):
    pass

### SLOO

In [ ]:
sample_XYs = XYs.sample(100000)

In [ ]:
cv = spacv.SKCV(folds= len(sample_XYs))

In [ ]:
%%timeit -r 1 -n 1
for tr,te in cv.split(sample_XYs):
    pass

### User defined polygons

In [ ]:
custom_example_poly = gpd.read_file('data/holes_example.geojson')
custom_example_pts = gpd.read_file('data/holes_example_pts.geojson')

In [ ]:
f, ax = plt.subplots(1,figsize=(6,4))
custom_example_pts.plot(ax=ax)
custom_example_poly.plot(ax=ax,alpha=.3, edgecolor='black')
custom_example_poly.sample(2, random_state=123).buffer(10).plot(ax=ax, alpha=.2), ax.axis('off');

In [ ]:
cv = spacv.UserDefinedSCV(custom_polygons = custom_example_poly, buffer_radius=10)

In [ ]:
%%timeit
for tr,te in cv.split(custom_example_pts):
    pass

# Ad-hoc grid/fold construction

In [ ]:
f, ax = plt.subplots(1,2, figsize=(16,16))
construct_blocks(XYs, tiles_x=5, tiles_y=3, method='unique').plot(column='grid_id',
                                edgecolor='black', cmap='tab20',ax=ax[0])

hexgrid = construct_blocks(XYs, tiles_x=5, tiles_y=2, 
                 method='optimized_random', n_sims=10, data=X,
                 n_groups=5, shape='hex')

hexgrid.plot(column='grid_id', cmap='tab20c',ax=ax[1], edgecolor='black')
XYs.plot(ax=ax[0], color='black'),XYs.plot(ax=ax[1], color='black')
ax[0].axis('off'), ax[1].axis('off')
plt.show()

In [ ]:
f, ax = plt.subplots(1,2, figsize=(14,12))

construct_blocks(XYs, method='systematic', tiles_x=15, tiles_y=5, direction='anti').plot(column='grid_id', cmap='Set1', 
                                                                                         ax=ax[0], edgecolor='black')
construct_blocks(XYs, method='systematic', tiles_x=15, tiles_y=5, direction='diagonal').plot(column='grid_id', cmap='Set1', 
                                                                                             ax=ax[1], edgecolor='black')
XYs.plot(ax=ax[0], color='black')
XYs.plot(ax=ax[1], color='black')
ax[0].axis('off'), ax[1].axis('off');

In [ ]:
f, ax = plt.subplots(1,3, figsize=(18,20))
construct_blocks(df, method='random', tiles_x=10, tiles_y=10, n_groups=3).plot(column='grid_id', ax=ax[0], edgecolor='black', cmap='viridis')
construct_blocks(df, method='random', tiles_x=10, tiles_y=10, n_groups=3).plot(column='grid_id', ax=ax[1], edgecolor='black', cmap='viridis')
construct_blocks(df, method='random', tiles_x=10, tiles_y=10, n_groups=3).plot(column='grid_id', ax=ax[2], edgecolor='black', cmap='viridis')
XYs.plot(ax=ax[0], color='black')
XYs.plot(ax=ax[1], color='black')
XYs.plot(ax=ax[2], color='black')
ax[0].axis('off'), ax[1].axis('off'), ax[2].axis('off');

In [ ]:
f, ax = plt.subplots(1, figsize=(8,8))
construct_blocks(df, method='optimized_random', data=X, tiles_x=25, tiles_y=25, 
                 n_groups=3, n_sims=10, distance_metric='euclidean').plot(column='grid_id', ax=ax, cmap='viridis', edgecolor='black')
df.plot(ax=ax, color='black')
ax.axis('off');

# Visualisaton

In [ ]:
df = gpd.read_file('/opt/conda/lib/python3.7/site-packages/libpysal/examples/baltim/baltim.shp')

XYs = df['geometry']
cols = ['NROOM', 'PRICE', 'AGE', 'SQFT']
X = df[cols]
y = df['PATIO']

In [ ]:
lags = np.arange(0, y_suggested, 20)
bw = 10

f, ax = plot_autocorrelation_ranges(XYs, X[cols], lags, bw)

In [ ]:
df['demo_labs'] = KMeans(6).fit(geometry_to_2d(df.geometry)).labels_
new_data = df[df['demo_labs'] == 1].copy()
training_data = df[df['demo_labs'] != 1]
columns = ['NROOM', 'DWELL', 'NBATH', 'PRICE', 'LOTSZ', 'SQFT']

In [ ]:
f, ax = plot_aoa(new_data, training_data, columns, (14,4))

# Future

* Spatially-explicit GCV
* Bootstrap samples from within folds testing